Import:

In [ ]:
import pandas as pd

Caricamento del dataset:

In [ ]:
df = pd.read_csv("./data/monsters.csv")

### Rimozione della colonna `desc`

In [ ]:
# Verifica se la colonna esiste prima di rimuoverla per evitare errori
if 'desc' in df.columns:
    df = df.drop(columns=['desc'])
    print("Colonna 'desc' rimossa con successo.")
else:
    print("La colonna 'desc' non è stata trovata (forse è già stata rimossa).")

# Verifica rapida
print("Colonne rimaste:", df.columns.tolist())

### --- LOGICA DI AGGREGAZIONE ---

La logica richiesta è:
Group (Priorità 1) -> Subtype (Priorità 2) -> Type(Fallback)

- step 1: prendo la colonna group come base

In [ ]:
df['sub_race'] = df['group']

 - Step 2: Riempiamo i buchi (NaN) di 'sub_race' con i valori di 'subtype'

 Questo rispetta la direttiva: "se group c'è, lo tengo. Se manca, guardo subtype"

In [ ]:
df['sub_race'] = df['sub_race'].fillna(df['subtype'])

- step 3: Riempiamo i buchi rimanenti con i valori di 'type'

Questo rispetta la direttiva: "se non sono presenti nessuno dei due, copia type"

In [ ]:
df['sub_race'] = df['sub_race'].fillna(df['type'])

### --- VERIFICA DEL RISULTATO ---
Visualizziamo alcuni esempi per verificare che la logica funzioni

In [ ]:
print("--- Esempio: Priorità Group (Entrambi presenti) ---")
# Mostri che hanno sia gruppo che sottotipo (es. Diavoli)
print(df[df['group'].notnull() & df['subtype'].notnull()][['name', 'group', 'subtype', 'sub_race']].head(3))

print("\n--- Esempio: Solo Subtype ---")
# Mostri umanoidi generici
print(df[df['group'].isnull() & df['subtype'].notnull()][['name', 'group', 'subtype', 'sub_race']].head(3))

print("\n--- Esempio: Fallback su Type (Nessuno presente) ---")
# Mostri senza sottocategorie specifiche
print(df[df['group'].isnull() & df['subtype'].isnull()][['name', 'type', 'sub_race']].head(3))

Ora rimuoviamo le feature `subtype` e `group` che adesso non sono più necessarie

In [ ]:
if 'sub_race' in df.columns:
    # Rimuove le colonne se presenti, ignorando errori se non ci sono
    df = df.drop(columns=['subtype', 'group'], errors='ignore')

    # Verifica finale: dovrebbe esserci solo 'sub_race' ma non più 'subtype' o 'group'
    print("Colonne attuali nel dataset:", df.columns.tolist())
else:
    print("Cancellazione non consentita")

### Riordinamento delle colonne


In [ ]:
# Spostiamo 'sub_race' esattamente dopo 'type'

cols = df.columns.tolist()

# Verifichiamo che entrambe le colonne esistano per evitare errori
if 'sub_race' in cols and 'type' in cols:
    # Rimuoviamo 'sub_race' dalla sua posizione attuale (che è in fondo)
    cols.remove('sub_race')

    # Troviamo l'indice della colonna 'type'
    type_index = cols.index('type')

    # Inseriamo 'sub_race' nella posizione successiva (+1) rispetto a 'type'
    cols.insert(type_index + 1, 'sub_race')

    # Applichiamo il nuovo ordine al DataFrame
    df = df[cols]
    print("Colonne riordinate con successo: 'sub_race' è ora accanto a 'type'.")

    # Verifica visiva
    print(df.columns.tolist()[:10]) # Stampa le prime 10 colonne per controllo
else:
    print("Attenzione: Impossibile riordinare. Colonne 'type' o 'sub_race' mancanti.")

### Salvataggio delle modifiche su file
Sovrascriviamo il file o ne creiamo uno nuovo (consigliato uno nuovo per sicurezza)

In [ ]:
output_path = "./data/monsters.csv"
df.to_csv(output_path, index=False)

print(f"\nSalvataggio completato! Il file modificato si trova in: {output_path}")